In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
import re
import mysql.connector

In [3]:
browser = webdriver.Edge('C:/Users/ace19/Documents/edgedriver_win32_ver98/msedgedriver.exe') # my version 98.0.1108.50
browser.get('http://www.kingbus.com.tw/ticketRoute.php')

C:\Users\ace19\AppData\Local\Temp/ipykernel_15352/3042824412.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Edge('C:/Users/ace19/Documents/edgedriver_win32_ver98/msedgedriver.exe') # my version 98.0.1108.50


In [4]:
start = Select(browser.find_element(by='xpath',value='//*[@id="area"]'))
start.select_by_value('台北')
time.sleep(0.1)
up = Select(browser.find_element(by='xpath',value='//*[@id="origin"]'))
up.select_by_value('臺北轉運站')
time.sleep(0.1)
down = Select(browser.find_element(by='xpath',value='//*[@id="destination"]'))
down.select_by_value('朝馬轉運站')
time.sleep(0.1)

In [5]:
search_buttom = browser.find_element(by='xpath', value='//*[@id="wrapper"]/div[2]/div/div[2]/input[1]')
search_buttom.click()

In [6]:
bsobj = BeautifulSoup(browser.page_source)

In [7]:
parent_myobj = bsobj.find(name='ul',attrs={'class':'routeData'})
myobj = parent_myobj.find_all('a')

In [8]:
del_pattern = re.compile('\s*')

In [9]:
myobj = [del_pattern.sub('',i.text) for i in myobj]

In [10]:
pattern = re.compile('【.*\[')
result = []
for i in myobj:
    t = pattern.search(i)
    start, end = t.span()
    first_str = i[start:end-1]
    second_str = i[end-1:]
    one_item = (first_str, second_str)
    print(one_item)
    result.append(one_item)

('【1827】臺北－朝馬－臺中', '[臺北轉運站到國光客運臺中車站]')
('【1831】臺北－南投', '[臺北轉運站到南投站]')
('【1832】臺北－埔里', '[臺北轉運站到埔里站](台14線)')
('【1832A】臺北－埔里', '[臺北轉運站到埔里站](國六)')
('【1832B】臺北－普台高中[經國道6號]', '[臺北轉運站到普台高中](國六、埔里)')
('【1837】臺北－臺南', '[臺北轉運站到臺南轉運站](林口朝馬)')
('【1837】臺北－臺南(不下林口)', '[臺北轉運站到臺南轉運站](朝馬)')
('【1838】臺北－高雄', '[臺北轉運站到高雄站](林口朝馬)')
('【1838】臺北—高雄(不下林口)', '[臺北轉運站到高雄站](朝馬)')
('【1839】臺北－屏東', '[臺北轉運站到麟洛站(屏東)](朝馬)')
('【1839】臺北－屏東(縮駛屏東轉運站)', '[臺北轉運站到屏東轉運站](朝馬)')


In [11]:
browser.close()

In [12]:
import pandas as pd

In [13]:
df = pd.DataFrame(result, columns=['route', 'desciipition'])

In [14]:
df.to_csv('route.csv',sep='\t', encoding='utf-8', index=False)

# csv to sql

In [15]:
connection = mysql.connector.connect(host='localhost',
                                       port='3306',
                                       user='root',
                                       password='271828',
                                       buffered = True, )

In [16]:
cursor = connection.cursor()

In [18]:
# cursor.execute('CREATE DATABASE `route`;')
cursor.execute('SHOW DATABASES;')

In [21]:
cursor.execute('USE `route`;')
cursor.execute('''
                CREATE TABLE route_list (
                    `ID` INT PRIMARY KEY, 
                    `route` VARCHAR(30) ,
                    `desciipition` VARCHAR(30)
                    )
               ''')

In [35]:
for idx, row in df.iterrows():
    route, desciipition = row
    sql = "INSERT INTO `route_list` (ID, route, desciipition) VALUES (%s, %s, %s)"
    val = (idx, route, desciipition)
    cursor.execute(sql, val)

In [45]:
cursor.execute("select * from route_list")
myresult = cursor.fetchall()

for x in myresult:
      print(x)

(0, '【1827】臺北－朝馬－臺中', '[臺北轉運站到國光客運臺中車站]')
(1, '【1831】臺北－南投', '[臺北轉運站到南投站]')
(2, '【1832】臺北－埔里', '[臺北轉運站到埔里站](台14線)')
(3, '【1832A】臺北－埔里', '[臺北轉運站到埔里站](國六)')
(4, '【1832B】臺北－普台高中[經國道6號]', '[臺北轉運站到普台高中](國六、埔里)')
(5, '【1837】臺北－臺南', '[臺北轉運站到臺南轉運站](林口朝馬)')
(6, '【1837】臺北－臺南(不下林口)', '[臺北轉運站到臺南轉運站](朝馬)')
(7, '【1838】臺北－高雄', '[臺北轉運站到高雄站](林口朝馬)')
(8, '【1838】臺北—高雄(不下林口)', '[臺北轉運站到高雄站](朝馬)')
(9, '【1839】臺北－屏東', '[臺北轉運站到麟洛站(屏東)](朝馬)')
(10, '【1839】臺北－屏東(縮駛屏東轉運站)', '[臺北轉運站到屏東轉運站](朝馬)')


In [36]:
connection.commit()

# 計數埔里在 route 的資料

In [37]:
connection = mysql.connector.connect(host='localhost',
                                       port='3306',
                                       user='root',
                                       password='271828',
                                       buffered = True, )
cursor = connection.cursor()

In [38]:
cursor.execute('USE `route`;')

In [43]:
cursor.execute("select count(*) from `route_list` where route like '%埔里%'")

In [44]:
myresult = cursor.fetchall()

for x in myresult:
      print(x)

(2,)
